# Imports

In [1]:
import pyedflib
import numpy as np
from pylab import *
from scipy import signal
from scipy.signal import welch, butter, filtfilt, lfilter
import pandas as pd
import seaborn
import glob, os
from collections import OrderedDict
from scipy.stats import *
import csv
from statsmodels.tsa.holtwinters import SimpleExpSmoothing, Holt
import skimage

In [2]:
basedir = os.getcwd()
datasetdir = basedir + '\DatasetsPrediction'
# Put here the directory of the CHBMIT DATABASE
dbdir = r"F:\Master\TFM\chb-mit-scalp-eeg-database-1.0.0"

In [3]:
basedir

'C:\\Users\\Mario\\Documents\\Master\\TFM\\Codigo\\Jupiter\\EEG_ML\\DataSetCreation'

# Functions

### Reading functions

In [4]:
class Register:
    
    def __init__(self, name, fs, nseizures):
        self.name = name
        self.fs = fs
        self.nseizures = nseizures
        self.seizures = []
        self.channels = []
        self.ictaltime = 0
            
    def addSeizure (self, start, end):
        self.ictaltime += end - start
        seizure = [start, end]
        self.seizures.append(seizure)
        

def read_data(filename, channels=[]):
    f = pyedflib.EdfReader(filename)
    
    # if no channels are passed to the function
    if len(channels) == 0:
        channels = f.getSignalLabels()

    channel_names = f.getSignalLabels()
    fs = f.getSampleFrequencies()

    data = np.zeros((len(channels), f.getNSamples()[0]))
    for i, channel in enumerate(channels):
        data[i, :] = f.readSignal(channel_names.index(channel))
        
    time = np.linspace(0, data.shape[1]/fs[0], data.shape[1])
    f._close()
    return data, fs[0], time

def trunc(data, timeW, fs):
    samples = data.shape[1]
    N = timeW*fs
    nw = int(samples//N)

    data = data [:, 0:nw*N]
    time = np.linspace(0, data.shape[1]/fs, data.shape[1])
    return data, time, nw, N

def create_seizure_vector(seizures):
    seizure = zeros(samples)
    ict_end = 0
    for n in range (len(seizures)):
        pre_start = max(((seizures[n][0]- 300)*fs), 0, ict_end)   
        ict_start = seizures[n][0]*fs
        ict_end = seizures[n][1]*fs
        seizure[pre_start:ict_start] = np.ones(ict_start-pre_start)
        seizure[ict_start:ict_end] = np.ones(ict_end-ict_start) + 1

    seizureW = skimage.util.view_as_windows(seizure, N, N//2)
    seizureW = np.around(mean(seizureW, 1))
    return seizureW

### Processing functions

In [5]:
def highpass(s, cutoff, fs, order = 5):
    b, a = signal.butter(order, cutoff / (fs/2), btype='high', analog=False)
    return filtfilt(b, a, s, axis = 1)

def lowpass(s, f, fs, order=5):
    b, a = signal.butter(order, f / (fs/2))
    return lfilter(b, a, s, axis = 1)

def frec2sample_range(fi, fo, fs, N):
    si = max(1,floor(fi*N/fs))
    so = ceil(fo*N/fs)
    return int(si), int(so)

def band_energy(fft, fs):
    N = len(fft)
    # Energia total
    et = sum(fft)
    dsi, dso = frec2sample_range(0.5, 4, fs, N)
    d = sum(fft[dsi:dso])
    tsi, tso = frec2sample_range(4, 7, fs, N)
    t = sum(fft[tsi:tso])
    asi, aso = frec2sample_range(7, 13, fs, N)
    a = sum(fft[asi:aso])
    bsi, bso = frec2sample_range(13, 30, fs, N)
    b = sum(fft[bsi:bso])
    gsi, gso = frec2sample_range(30, 50, fs, N)
    g = sum(fft[gsi:gso])
    return et, d, t, a, b, g

def psd(signal):
    X = fft(signal)
    Px = real(X*conj(X))
    return Px

def exponential_smooth(timeseries, alpha=0.3):
    model = SimpleExpSmoothing(timeseries)
    fit = model.fit(smoothing_level=alpha)
    return fit.fittedvalues

def spectral_centroid(nf, ps):
    return sum(nf * ps)

def variational_coeff(nf, ps, sc):
    return sum( (((nf - sc))**2) * ps) / sum(ps)

def spectral_skew(nf, ps, sc, vc):
    return sum( ((nf - sc)/vc)**3 * ps) / sum(ps)
   
def bandpower(freqs, psd, band, output = False):
    band = np.asarray(band)
    low, high = band
    
    # Find closest indices of band in frequency vector
    idx_min = np.argmax(np.round(freqs) > low) - 1
    idx_max = np.argmax(np.round(freqs) > high)
    # select frequencies of interest
    psd = sum(psd[idx_min:idx_max])

    return psd

def power_measures(data, sample_rate, output=False):
    bandpasses = [[[0.5,50],'power'],
                  [[0.5,4],'power_delta'],
                  [[4,8],'power_theta'],
                  [[8,13],'power_alpha'],
                  [[13,30],'power_beta'],
                  [[30,50],'power_gamma']]
    
    # Compute the periodogram (Welch)
    freqs, psd = welch(data, sample_rate, nperseg=(sample_rate*2), window='hamm', scaling='density', axis=0)
    
    bandpass_data = np.zeros(len(bandpasses))
    for i, [bandpass, freq_name] in enumerate(bandpasses):
        bandpass_data[i] = bandpower(freqs, psd, bandpass)
    return bandpass_data   
    

def channel_processing(channel_matrix, fs):
    
        ninstances = channel_matrix.shape[0]
        power_bands = zeros([ninstances, 6])

        meanv = zeros(ninstances)
        variancev = zeros(ninstances)
        skewnessv = zeros(ninstances)
        kurtosisv = zeros(ninstances)
        stdv = zeros(ninstances)
        zcrossingsv = zeros(ninstances)
        p2pv = zeros(ninstances)

        features = ['mean', 'variance', 'skewness', 'kurtosis', 'std', 'zero_crossings', 'peak2peak',
                    'total_energy', 'delta', 'theta', 'alpha', 'beta', 'gamma']

        for index, row in enumerate(channel_matrix):
            power_bands[index, :] = power_measures(row, fs)
            
            try:
                meanv[index] = mean(row)
                variancev[index] = var(row)
                skewnessv[index] = skew(row)
                kurtosisv[index] = kurtosis(row)
                stdv[index] = std(row)
                zcrossingsv[index] = len(np.where(np.diff(np.sign(row)))[0])
                p2pv[index] = max(row)-min(row)

            except ZeroDivisionError:
                meanv[index] = 0.001
                variancev[index] = 0.001
                skewnessv[index] = 0.001
                kurtosisv[index] = 0.001
                stdv[index] = 0.001
                zcrossingsv[index] = 0.001
                p2pv[index] = 0.001

        data = [(meanv),(variancev),   (skewnessv),
                  (kurtosisv),   (stdv),  (zcrossingsv),  (p2pv),
                  (power_bands[:, 0]),   (power_bands[:, 1]),   (power_bands[:, 2]),
                  (power_bands[:, 3]),   (power_bands[:, 4]),  (power_bands[:, 5])
        ]
        

        data = np.array(data).transpose()
        df = pd.DataFrame(data, columns = features)
        
        return df

In [6]:
def read_annotations(annotation):
    with open(annotation) as f:
        registers = {}
        channels_dict = {}
        nmontages = 1
        for line in f:
            if ("Data Sampling Rate" in line):
                line = line.split()
                fs = int(line[3]) 
                
            if ("Channel " in line):
                line = line.split()
                channel = line[2]
                if channel in channels_dict:
                    channels_dict.update({channel: channels_dict[channel]+1})
                else:
                    channels_dict[channel] = 1

            if ("Channels changed" in line):
                nmontages += 1
                
            elif ("File Name" in line):
                name = line.split()[2]
                while True:
                    newLine = f.readline()
                    if ("Number of Seizures" in newLine):
                        nseizures = int(newLine.split()[5])
                        register = Register(name, fs, nseizures)
                        if nseizures > 0:
                            for i in range(nseizures):
                                line1 = f.readline().split()
                                line2 = f.readline().split()
                                if (line1[3] == "Time:"):
                                    start = int(line1[4])
                                    end = int(line2[4])
                                else:
                                    start = int(line1[3])
                                    end = int(line2[3])
                                    
                                register.addSeizure(start, end)

                        registers[name] = register
                        break
    common_channels = []
    [common_channels.append(key) for key in channels_dict.keys() if channels_dict[key] == nmontages]
    channel_index = dict(zip( list(np.arange(len(common_channels))), common_channels ))
    return registers, channel_index

# Create Dataset for only one patient

In [10]:
patient = 'chb01'

In [11]:
f = open(dbdir + '\RECORDS-WITH-SEIZURES', 'r', encoding = 'utf-8')
seizure_files = f.read().split('\n')
seizure_files = list(map(lambda string: string[6:], seizure_files))
f.close()

fdir = dbdir + '\\' + patient

In [12]:
os.chdir(fdir)
annotation = glob.glob('*txt')

registers, channel_index = read_annotations(annotation[0])
total_ictaltime = 0
for key, value in registers.items():
    total_ictaltime += value.ictaltime
channel_index

{0: 'FP1-F7',
 1: 'F7-T7',
 2: 'T7-P7',
 3: 'P7-O1',
 4: 'FP1-F3',
 5: 'F3-C3',
 6: 'C3-P3',
 7: 'P3-O1',
 8: 'FP2-F4',
 9: 'F4-C4',
 10: 'C4-P4',
 11: 'P4-O2',
 12: 'FP2-F8',
 13: 'F8-T8',
 14: 'P8-O2',
 15: 'FZ-CZ',
 16: 'CZ-PZ',
 17: 'P7-T7',
 18: 'T7-FT9',
 19: 'FT9-FT10',
 20: 'FT10-T8'}

In [25]:
patient_seizure_files = []
patient_non_seizure_files = []

patient_files = sorted(registers.keys())
for file in patient_files:
    if file in seizure_files:
        patient_seizure_files.append(file)
    else:
        patient_non_seizure_files.append(file)
        
patient_non_seizure_files = patient_non_seizure_files[-min(4, len(patient_seizure_files)):]
print(patient_seizure_files)
print(patient_non_seizure_files)

['chb01_03.edf', 'chb01_04.edf', 'chb01_15.edf', 'chb01_16.edf', 'chb01_18.edf', 'chb01_21.edf', 'chb01_26.edf']
['chb01_41.edf', 'chb01_42.edf', 'chb01_43.edf', 'chb01_46.edf']


In [27]:
# Select window duration in seconds
timeW = 4
nchannels = len(channel_index)
decimationCoeff = 2
fs = registers['chb01_01.edf'].fs

selected_channels_lof = []

dataframe = pd.DataFrame()
for key, value in registers.items():
    
    patient_seizure_files = []
    patient_non_seizure_files = []
    patient_files = sorted(registers.keys())
    for file in patient_files:
        if file in seizure_files:
            patient_seizure_files.append(file)
        else:
            patient_non_seizure_files.append(file)
    patient_non_seizure_files = patient_non_seizure_files[-min(4, len(patient_seizure_files)):]

    # Signal reading: only if is a seizure file
    if key in patient_seizure_files:
        signals, originalfs, time = read_data(key, channel_index.values())
        # Decimation
        signals = signal.decimate(signals, decimationCoeff)
        fs = originalfs//decimationCoeff
        # Filtering 
        signals = highpass(signals, 0.5, fs)
        signals = lowpass(signals, 49, fs)
        
        # Truncate to generate time windows
        signals_trunc, time, nw, N = trunc(signals, timeW, fs)
        samples = signals_trunc.shape[1]

        print("Readed " + key)

        # Seizure vector creation
        seizureW = create_seizure_vector(value.seizures)     

        # Create register dataframe
        auxdf = pd.DataFrame()
        for channel, s in enumerate(signals_trunc):
            # Only windows with preictal and ictal content
            newSignal = skimage.util.view_as_windows(s, N, N//2)
            used_index = np.where((seizureW == 2) | (seizureW == 1))[0]
            newSignal = newSignal[used_index, :]
            newdf = channel_processing(newSignal, fs)
            # Get new seizure vector renamed
            aux = seizureW[used_index]
            seizureW_new = ['pre-ictal' if x == 1 else 'ictal' for x in aux]
            # Create aux dataframe
            newdf['channel'] = pd.Series( [channel_index[channel]]*len(used_index), index = newdf.index)
            newdf['seizure'] = pd.Series( seizureW_new, index = newdf.index)
            auxdf = auxdf.append(newdf, ignore_index=True)

        # Add to the patient dataframe
        dataframe = dataframe.append(auxdf, ignore_index=True)
        print("Rows created for " + key)
        
    elif key in patient_non_seizure_files:
        signals, originalfs, time = read_data(key, channel_index.values())
        # Decimation
        signals = signal.decimate(signals, decimationCoeff)
        fs = originalfs//decimationCoeff
        # Filtering 
        signals = highpass(signals, 0.5, fs)
        signals = lowpass(signals, 49, fs)

        # Truncate to generate time windows
        signals_trunc, time, nw, N = trunc(signals, timeW, fs)
        samples = signals_trunc.shape[1]
        
        print("Readed " + key)

        # Create register dataframe
        auxdf = pd.DataFrame()
        for channel, s in enumerate(signals_trunc):
            newSignal = skimage.util.view_as_windows(s, N, N//2)
            nw = newSignal.shape[0]
            newdf = channel_processing(newSignal, fs)
            newdf['channel'] = pd.Series( [channel_index[channel]]*nw, index = newdf.index)
            newdf['seizure'] = pd.Series( ['inter-ictal']*nw, index = newdf.index)
            auxdf = auxdf.append(newdf, ignore_index=True)

        # Add to the patient dataframe
        dataframe = dataframe.append(auxdf, ignore_index=True)
        print("Rows created for " + key)
        
# Save the datase and the csv with the list of significant channels
dataframe.to_hdf(datasetdir + '\\' + patient + 'features' + '.h5', key = 'fullpatient', mode = 'w', format = 'table')
        

os.chdir(basedir)

Readed chb01_03.edf
Rows created for chb01_03.edf
Readed chb01_04.edf
Rows created for chb01_04.edf
Readed chb01_15.edf
Rows created for chb01_15.edf
Readed chb01_16.edf
Rows created for chb01_16.edf
Readed chb01_18.edf
Rows created for chb01_18.edf
Readed chb01_21.edf
Rows created for chb01_21.edf
Readed chb01_26.edf
Rows created for chb01_26.edf
Readed chb01_41.edf
Rows created for chb01_41.edf
Readed chb01_42.edf
Rows created for chb01_42.edf
Readed chb01_43.edf
Rows created for chb01_43.edf
Readed chb01_46.edf
Rows created for chb01_46.edf


OSError: ``F:\Master\TFM\chb-mit-scalp-eeg-database-1.0.0\chb01\DatasetsPrediction`` does not exist

In [31]:
#dataframe = dataframe[(dataframe['channel'] == 'FT9-FT10')]
#dataframe = dataframe.drop(['channel'], axis=1)
dataframe[dataframe['seizure'] == 'pre-ictal']

,mean,variance,skewness,kurtosis,std,zero_crossings,peak2peak,total_energy,delta,theta,alpha,beta,gamma,seizure
3230,0.863884,967.215819,-0.274751,0.950084,31.100094,68.0,216.930536,2211.712213,1432.522747,590.705214,46.333634,111.096326,86.021547,pre-ictal
3231,0.000695,1245.484248,0.033599,0.504990,35.291419,59.0,223.218288,2092.886602,1455.452374,400.917103,97.994140,118.624553,83.486632,pre-ictal
3232,-0.224911,824.826713,0.379148,0.834155,28.719797,83.0,192.156440,1622.981769,1011.274686,382.168678,99.828357,142.686062,86.662286,pre-ictal
3233,-0.054347,606.052576,-0.143503,0.041334,24.618135,86.0,137.980638,1196.738452,741.215130,271.000524,86.539267,119.733086,90.303645,pre-ictal
3234,-0.494037,531.267935,-0.187822,0.385579,23.049250,95.0,137.980638,1056.414807,694.134286,226.529560,49.391451,94.713310,80.992746,pre-ictal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26456,0.489275,811.925790,-0.064299,0.607260,28.494312,66.0,183.566006,1416.604589,911.829500,289.597955,122.931876,110.100569,43.026371,pre-ictal
26457,-0.394202,1101.072061,-0.054281,0.375256,33.182406,69.0,214.192239,2226.749932,1414.661995,452.683849,254.374482,132.563400,42.123030,pre-ictal
26458,0.394619,1174.411955,0.112059,-0.074020,34.269694,67.0,203.507787,2377.573015,1321.741077,652.285626,271.171343,173.049564,39.255257,pre-ictal
26459,0.309318,863.863571,0.024794,-0.325381,29.391556,66.0,162.688533,1784.306444,947.493461,518.585085,154.581701,238.145508,33.946853,pre-ictal


# Dataset creation for all the patients

In [7]:
os.chdir(dbdir)
patients = [name for name in os.listdir(".") if os.path.isdir(name)]
#patients = patients[11:]
patients.remove('chb12')

In [8]:
f = open(dbdir + '\RECORDS-WITH-SEIZURES', 'r', encoding = 'utf-8')
seizure_files = f.read().split('\n')
seizure_files = list(map(lambda string: string[6:], seizure_files))
f.close()

In [9]:
timeW = 4
decimationCoeff = 2

for patient in patients:
    print('---------------------------------------------- Patient: ' + patient + ' ----------------------------------------------------')
    fdir = dbdir + '\\' + patient
    os.chdir(fdir)
    annotation = glob.glob('*txt')
    
    registers, channel_index = read_annotations(annotation[0])

    nchannels = len(channel_index)
    selected_channels_lof = []

    
    dataframe = pd.DataFrame()
    for key, value in registers.items():

        patient_seizure_files = []
        patient_non_seizure_files = []
        patient_files = sorted(registers.keys())
        for file in patient_files:
            if file in seizure_files:
                patient_seizure_files.append(file)
            else:
                patient_non_seizure_files.append(file)
        patient_non_seizure_files = patient_non_seizure_files[-min(4, len(patient_seizure_files)):]

        # Signal reading: only if is a seizure file
        if key in patient_seizure_files:
            signals, originalfs, time = read_data(key, channel_index.values())
            # Decimation
            signals = signal.decimate(signals, decimationCoeff)
            fs = originalfs//decimationCoeff
            # Filtering 
            signals = highpass(signals, 0.5, fs)
            signals = lowpass(signals, 49, fs)
            

            # Truncate to generate time windows
            signals_trunc, time, nw, N = trunc(signals, timeW, fs)
            samples = signals_trunc.shape[1]

            print("Readed " + key)

            # Seizure vector creation
            seizureW = create_seizure_vector(value.seizures)     

            # Create register dataframe
            auxdf = pd.DataFrame()
            for channel, s in enumerate(signals_trunc):
                # Only windows with preictal and ictal content
                newSignal = skimage.util.view_as_windows(s, N, N//2)
                used_index = np.where((seizureW == 2) | (seizureW == 1))[0]
                newSignal = newSignal[used_index, :]
                newdf = channel_processing(newSignal, fs)
                # Get new seizure vector renamed
                aux = seizureW[used_index]
                seizureW_new = ['pre-ictal' if x == 1 else 'ictal' for x in aux]
                # Create aux dataframe
                newdf['channel'] = pd.Series( [channel_index[channel]]*len(used_index), index = newdf.index)
                newdf['seizure'] = pd.Series( seizureW_new, index = newdf.index)
                auxdf = auxdf.append(newdf, ignore_index=True)

            # Add to the patient dataframe
            dataframe = dataframe.append(auxdf, ignore_index=True)
            print(dataframe.shape)
            print("Rows created for " + key)

        elif key in patient_non_seizure_files:
            signals, originalfs, time = read_data(key, channel_index.values())
            # Decimation
            signals = signal.decimate(signals, decimationCoeff)
            fs = originalfs//decimationCoeff
            # Filtering 
            signals = highpass(signals, 0.5, fs)
            signals = lowpass(signals, 49, fs)

            # Truncate to generate time windows
            signals_trunc, time, nw, N = trunc(signals, timeW, fs)
            samples = signals_trunc.shape[1]

            print("Readed " + key)

            # Create register dataframe
            auxdf = pd.DataFrame()
            for channel, s in enumerate(signals_trunc):
                newSignal = skimage.util.view_as_windows(s, N, N//2)
                nw = newSignal.shape[0]
                newdf = channel_processing(newSignal, fs)
                newdf['channel'] = pd.Series( [channel_index[channel]]*nw, index = newdf.index)
                newdf['seizure'] = pd.Series( ['inter-ictal']*nw, index = newdf.index)
                auxdf = auxdf.append(newdf, ignore_index=True)

            # Add to the patient dataframe
            dataframe = dataframe.append(auxdf, ignore_index=True)
            print("Rows created for " + key)

    # Save the datase and the csv with the list of significant channels
    dataframe.to_hdf(datasetdir + '\\' + patient + 'features' + '.h5', key = 'fullpatient', mode = 'w', format = 'table')
    

---------------------------------------------- Patient: chb01 ----------------------------------------------------
Readed chb01_03.edf
(3570, 15)
Rows created for chb01_03.edf
Readed chb01_04.edf
(7014, 15)
Rows created for chb01_04.edf
Readed chb01_15.edf
(10584, 15)
Rows created for chb01_15.edf
Readed chb01_16.edf
(14280, 15)
Rows created for chb01_16.edf
Readed chb01_18.edf
(18375, 15)
Rows created for chb01_18.edf
Readed chb01_21.edf
(22512, 15)
Rows created for chb01_21.edf
Readed chb01_26.edf
(26712, 15)
Rows created for chb01_26.edf
Readed chb01_41.edf
Rows created for chb01_41.edf
Readed chb01_42.edf
Rows created for chb01_42.edf
Readed chb01_43.edf
Rows created for chb01_43.edf
Readed chb01_46.edf
Rows created for chb01_46.edf
---------------------------------------------- Patient: chb02 ----------------------------------------------------
Readed chb02_16.edf
(2226, 15)
Rows created for chb02_16.edf
Readed chb02_19.edf
(5481, 15)
Rows created for chb02_19.edf
Readed chb02_34.

In [89]:
a = np.array([[0, 1, 2, 3, 4, 5, 6, 7, 8], [0, 1, 2, 3, 4, 5, 6, 7, 8]])
b = np.array([0, 0, 1, 1, 1, 0, 0, 0, 0])
a.shape

(2, 9)

In [81]:
np.where(b == 1)

(array([2, 3, 4], dtype=int64),)

In [90]:
c = a[:, np.where(b == 1)[0]]

In [91]:
c.shape

(2, 3)